<a href="https://colab.research.google.com/github/SARA3SAEED/abu-LLM/blob/main/5_abu_session_02_01_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers==3.0.1 datasets

## Download Dataset

In [ ]:
from datasets import load_dataset
from tqdm.auto import tqdm
import pandas as pd

In [ ]:
ds = load_dataset("Cohere/wikipedia-22-12", "ar", streaming=True, trust_remote_code=True)

In [ ]:
max_titles = 500
all_titles = set()
collected_recs = []

progress_bar = tqdm(total=max_titles)

for rec in ds["train"]:
    if rec["title"] not in all_titles:
        all_titles.add(rec["title"])
        progress_bar.update(1)

    collected_recs.append(rec)

    if len(all_titles) > max_titles:
        break

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
collected_df = pd.DataFrame(collected_recs)
collected_df.shape

(28692, 8)

In [ ]:
collected_df["text_len"] = collected_df["text"].apply( lambda text: len(text) )

In [ ]:
collected_df["text_len"].describe()

,text_len
count,28692.000000
mean,532.142897
std,376.670192
min,101.000000
25%,281.000000
50%,441.000000
75%,669.000000
max,5477.000000


In [ ]:
cleaned_collected_df = collected_df[ collected_df["text_len"] < 1500 ]
cleaned_collected_df["text_len"].describe()

,text_len
count,27946.000000
mean,493.219244
std,284.976974
min,101.000000
25%,278.000000
50%,431.000000
75%,645.000000
max,1499.000000


In [ ]:
cleaned_collected_df = cleaned_collected_df.sample(frac=1, random_state=101)

In [ ]:
cleaned_collected_df.head()

,id,title,text,url,wiki_id,views,paragraph_id,langs,text_len
2156,2156,إيلون ماسك,في 22 ديسمبر 2015، نجحت سبيس إكس في إعادة المر...,https://ar.wikipedia.org/wiki?curid=1280929,1280929,2492.724641,22,136,343
12507,12507,فهد بن عبد العزيز آل سعود,في محطته الثانية في منطقة مكة المكرمة في محافظ...,https://ar.wikipedia.org/wiki?curid=6588,6588,1884.711635,41,72,446
22403,22403,مايكل جاكسون,في عام 2016، قدرت فوربس إجمالي الأرباح السنوية...,https://ar.wikipedia.org/wiki?curid=27129,27129,1639.789786,114,252,457
21415,21415,القمر,تم قياس قوة مجال القمر المغناطيسي بواسطة دراسة...,https://ar.wikipedia.org/wiki?curid=1619,1619,1659.302521,20,268,725
4046,4046,صدام حسين,فالذين عرفوا إبراهيم الحسن يصفونه كما ذكر صلاح...,https://ar.wikipedia.org/wiki?curid=584101,584101,2315.063137,8,124,309


### Ollama Embeddings

In [ ]:
!pip install ollama==0.3.2
!curl https://ollama.ai/install.sh | sh

In [ ]:
ollama_model_id = "paraphrase-multilingual"

In [ ]:
# run ollama server
!nohup ollama serve &
!sleep 8 && tail nohup.out

In [ ]:
!ollama pull {ollama_model_id}

In [ ]:
!nohup ollama run {ollama_model_id} &
!sleep 8 && tail nohup.out

### Evaluate Ollama Embedding

In [ ]:
from sentence_transformers.util import semantic_search, cos_sim
import torch
import ollama

In [ ]:
corpus = cleaned_collected_df["text"].values[:50]

corpus_embeddings = []
for text in tqdm(corpus):
    corpus_embeddings.append(
        ollama.embeddings(model=ollama_model_id, prompt=text)["embedding"]
    )

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
question_text = "من مترجم كتابات ابن حيان؟"
question_embedding =  ollama.embeddings(
                        model='paraphrase-multilingual',
                        prompt=question_text,
                    )['embedding']

In [ ]:
question_embedding = torch.FloatTensor([question_embedding])
corpus_embeddings = torch.FloatTensor(corpus_embeddings)

In [ ]:
similar_documents = semantic_search(
    query_embeddings=question_embedding,
    corpus_embeddings=corpus_embeddings,
    top_k=10
)

In [ ]:
for document in similar_documents[0]:
    _id = document["corpus_id"]
    text = corpus[_id]
    score = document["score"]

    print("Score:", score)
    print("Text:", text)
    print("="*30)

Score: 0.5641813278198242
Text: أعطى الملحن اللبناني فيلمون وهبي (1918ـــ 1985) ألحانه لـ 27 أغنية من أغاني فيروز، منها 18 أغنية من تأليف وتوزيع وإشراف الأخوين رحباني غنتها فيروز في الفترة من 1962-1976، وثماني أغنيات لحنها لها فيلمون بعد انفصالها عن الأخوين رحباني غنتها فيروز في الفترة من 1980-1985، وجميعها من كلمات جوزيف حرب عدا واحدة فقط من كلمات طلال حيدر وهي أغنية (يا رايح)، ولا تزال هذه الأعمال حاضرة في الذاكرة اللبنانية والعربية فقد كان فيلمون هو الاستثناء الوحيد الذي تعاون معه الأخوين رحباني كملحن من خارج الأسرة الرحبانية، وقد اعترف منصور الرحباني بالنجاح الكبير الذي حققته ألحان فيلمون على حنجرة فيروز فيقول :.
Score: 0.5345969796180725
Text: وقد ذكرنا فيما تقدم أن الحجاج حج بالناس في هذه السنة الخارجة، وكان في الحج ابن عمر، وقد كتب عبد الملك إلى الحجاج أن يأتم بابن عمر في المناسك كما ثبت ذلك في الصحيحين.
Score: 0.529414176940918
Text: استطاع الفنانون الإيرانيون أن يبدعوا في الفن التصويري لمضامين المخطوطات الفارسية والعربية، كما نجحوا في تجويد الخط وتحسينه وتطويره، فقد امتاز الخط

## Test Sbert Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import semantic_search, cos_sim
import torch
from tqdm.auto import tqdm

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [ ]:
corpus = cleaned_collected_df["text"].values[:50]

corpus_embeddings = []
for text in tqdm(corpus):
    corpus_embeddings.append(
        model.encode(text).tolist()
    )

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
question_text = "من مترجم كتابات ابن حيان؟"
question_embedding =  model.encode(question_text).tolist()

In [ ]:
question_embedding = torch.FloatTensor([question_embedding])
corpus_embeddings = torch.FloatTensor(corpus_embeddings)

In [ ]:
similar_documents = semantic_search(
    query_embeddings=question_embedding,
    corpus_embeddings=corpus_embeddings,
    top_k=10
)

In [ ]:
for document in similar_documents[0]:
    _id = document["corpus_id"]
    text = corpus[_id]
    score = document["score"]

    print("Score:", score)
    print("Text:", text)
    print("="*30)

Score: 0.5286891460418701
Text: تعود شهرة جابر بن حيان إلى مؤلفاته العديدة، ومنها «كتاب الرسائل السبعين»، ترجمه إلى اللاتينية جيرار الكريموني سنة 1187م وتضاف إلى هذه الكتب تصانيف أخرى عديدة تتناول، إلى جانب الكيمياء، شروحاً لكتب أرسطو وأفلاطون؛ ورسائل في الفلسفة، والتنجيم، والرياضيات، الطب، والموسيقى. وجاء في «الأعلام» للزركلي أن جابراً له تصانيف كثيرة تتراوح ما بين مائتين واثنين وثلاثين (232) وخمسمائة (500) كتاب، لكن ضاع أكثرها. وقد ترجمت بعض كتب جابر إلى اللغة اللاتينية في أوائل القرن الثاني عشر، كما ترجم بعضها من اللاتينية إلى الإنجليزية عام 1678. وظل الأوربيون يعتمدون على كتبه لعدة قرون، وقد كان لها أثر كبير في تطوير الكيمياء الحديثة. وفي هذا يقول ماكس مايرهوف: يمكن إرجاع تطور الكيمياء في أوروبا إلى جابر ابن حيان بصورة مباشرة. وأكبر دليل على ذلك أن كثيراً من المصطلحات التي ابتكرها ما زالت مستعملة في مختلف اللغات الأوربية.
Score: 0.4976058006286621
Text: أعطى الملحن اللبناني فيلمون وهبي (1918ـــ 1985) ألحانه لـ 27 أغنية من أغاني فيروز، منها 18 أغنية من تأليف وتوزيع وإشراف الأخوين رح

### QDrant

In [ ]:
!pip install qdrant-client==1.11.1

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
import os
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import semantic_search, cos_sim
import torch
from tqdm.auto import tqdm

In [ ]:
qdrant_db_path = "./qdrant_db"
os.makedirs(qdrant_db_path, exist_ok=True)

qdrant_client = QdrantClient(path=qdrant_db_path)

In [ ]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [ ]:
collection_name = "ar_wiki_base"
vec_size = 384

qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=vec_size,
        distance=Distance.COSINE
    )
)

True

In [ ]:
cleaned_collected_df_sample = cleaned_collected_df.sample(5000, random_state=101)

In [ ]:
cleaned_collected_df_sample.head(1)

,id,title,text,url,wiki_id,views,paragraph_id,langs,text_len
11866,11866,الدولة الفاطمية,وفي الزمن المُعاصر، أصدت اللجنة الدائمة للبحوث...,https://ar.wikipedia.org/wiki?curid=1099980,1099980,1905.925263,77,70,508


In [ ]:
batch_size = 50
total = cleaned_collected_df_sample.shape[0]
rec_id = 0

for i in tqdm(range(0, total, batch_size)):

    batch_titles = cleaned_collected_df_sample["title"].values[i:i+batch_size]
    batch_texts = cleaned_collected_df_sample["text"].values[i:i+batch_size]

    batch_embeds = model.encode(batch_texts).tolist()

    points = []
    for title, text, embed in zip(batch_titles, batch_texts, batch_embeds):

        points.append(PointStruct(
            id=rec_id,
            vector=embed,
            payload={
                "title": title,
                "text": text
            }
        ))

        rec_id += 1

    qdrant_client.upsert(
        collection_name=collection_name,
        points=points
    )


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
query_text = "من كم مقطع تتألف اللغة في الصين؟"
query_embed = model.encode(query_text).tolist()

In [ ]:
query_embed[0]

0.1822662353515625

In [ ]:
hits = qdrant_client.search(
    collection_name=collection_name,
    query_vector=query_embed,
    limit=5
)

In [ ]:
hits

[ScoredPoint(id=10, version=0, score=0.7584870983372929, payload={'title': 'الصين', 'text': 'تمثل لغة قومية هان (الصينية المنطوقة والمكتوبة) اللغة الرسمية للبلاد، وهي تستخدم في كافة أنحائها، ومن حيث تعدادها تحتل هذه اللغة المرتبة الأولى في العالم، وتسمى محليًا «زونغ ون» (中文). رغم أن اللغة الصينية تشمل أكثر من 30 ألف مقطع (أو رمز) إلا أنه وحسب إحصاء المقاطع الصينية المكتوبة في الكتب والصحف الحديثة في الوقت الحاضر، يشكل حوالي 3000 مقطع صيني 99% من نسبة المقاطع الصينية المكتوبة المتكررة. ومن بين الخمسة وخمسون أقلية قومية، فإن قوميتا هوي والمانتشو تستخدمان اللغة الهانية، بينما تستخدم كل من القوميات الثلاثة وخمسون الأخرى لغتها الخاصة، ولإحدى وعشرين أقلية قومية لغتها المكتوبة. يتم تدريس اللغات القومية في المدارس والمناطق التي تسكنها هذه الأقليات.'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=2382, version=0, score=0.5513428739834357, payload={'title': 'أفغانستان', 'text': 'ملاحظة: تمثل البيانات اللغات الأكثر انتشارًا؛ يصل مجموع النسب إلى أكثر من 100٪ بسبب وجود الكثير من 

### Scalar Quantization

In [ ]:
from qdrant_client.models import ScalarQuantization, ScalarQuantizationConfig, ScalarType

# create new collection
collection_name = "ar_wiki_scalar_int8"
vec_size = 384

qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vec_size, distance=Distance.COSINE),
    quantization_config=ScalarQuantization(
        scalar=ScalarQuantizationConfig(
            type=ScalarType.INT8,
            quantile=0.95,
            always_ram=False,
        ),
    ),
)

True

In [ ]:
batch_size = 50
total = cleaned_collected_df_sample.shape[0]
rec_id = 0

for i in tqdm(range(0, total, batch_size)):

    batch_titles = cleaned_collected_df_sample["title"].values[i:i+batch_size]
    batch_texts = cleaned_collected_df_sample["text"].values[i:i+batch_size]

    batch_embeds = model.encode(batch_texts).tolist()

    points = []
    for title, text, embed in zip(batch_titles, batch_texts, batch_embeds):

        points.append(PointStruct(
            id=rec_id,
            vector=embed,
            payload={
                "title": title,
                "text": text
            }
        ))

        rec_id += 1

    qdrant_client.upsert(
        collection_name=collection_name,
        points=points
    )


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
query_text = "من كم مقطع تتألف اللغة في الصين؟"
query_embed = model.encode(query_text).tolist()

In [ ]:
hits = qdrant_client.search(
    collection_name=collection_name,
    query_vector=query_embed,
    limit=5
)

In [ ]:
hits

[ScoredPoint(id=10, version=0, score=0.7584870983372929, payload={'title': 'الصين', 'text': 'تمثل لغة قومية هان (الصينية المنطوقة والمكتوبة) اللغة الرسمية للبلاد، وهي تستخدم في كافة أنحائها، ومن حيث تعدادها تحتل هذه اللغة المرتبة الأولى في العالم، وتسمى محليًا «زونغ ون» (中文). رغم أن اللغة الصينية تشمل أكثر من 30 ألف مقطع (أو رمز) إلا أنه وحسب إحصاء المقاطع الصينية المكتوبة في الكتب والصحف الحديثة في الوقت الحاضر، يشكل حوالي 3000 مقطع صيني 99% من نسبة المقاطع الصينية المكتوبة المتكررة. ومن بين الخمسة وخمسون أقلية قومية، فإن قوميتا هوي والمانتشو تستخدمان اللغة الهانية، بينما تستخدم كل من القوميات الثلاثة وخمسون الأخرى لغتها الخاصة، ولإحدى وعشرين أقلية قومية لغتها المكتوبة. يتم تدريس اللغات القومية في المدارس والمناطق التي تسكنها هذه الأقليات.'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=2382, version=0, score=0.5513428739834357, payload={'title': 'أفغانستان', 'text': 'ملاحظة: تمثل البيانات اللغات الأكثر انتشارًا؛ يصل مجموع النسب إلى أكثر من 100٪ بسبب وجود الكثير من 

## Re-Ranking

In [ ]:
from transformers import AutoModelForSequenceClassification

reranker_model = AutoModelForSequenceClassification.from_pretrained(
    'jinaai/jina-reranker-v2-base-multilingual',
    torch_dtype="auto",
    trust_remote_code=True,
).to('cuda:0')

reranker_model.eval()

In [ ]:
query = "منتجات ازالة البقع السوداء تحت العينين"

documents = [
    "Organic skincare for sensitive skin with aloe vera and chamomile.",
    "New makeup trends focus on bold colors and innovative techniques",
    "Bio-Hautpflege für empfindliche Haut mit Aloe Vera und Kamille",
    "Produits pour éliminer les taches brunes sous les yeux",
    "Neue Make-up-Trends setzen auf kräftige Farben und innovative Techniken",
    "Cuidado de la piel orgánico para piel sensible con aloe vera y manzanilla",
    "Las nuevas tendencias de maquillaje se centran en colores vivos y técnicas innovadoras",
    "针对敏感肌专门设计的天然有机护肤产品",
    "新的化妆趋势注重鲜艳的颜色和创新的技巧",
    "敏感肌のために特別に設計された天然有機スキンケア製品",
    "新しいメイクのトレンドは鮮やかな色と革新的な技術に焦点を当てています",
]

sentence_pairs = [[query, doc] for doc in documents]

scores = reranker_model.compute_score(sentence_pairs, max_length=1024)

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [ ]:
query_text = "من كم مقطع تتألف اللغة في الصين؟"
query_embed = model.encode(query_text).tolist()

collection_name = "ar_wiki_scalar_int8"

hits = qdrant_client.search(
    collection_name=collection_name,
    query_vector=query_embed,
    limit=10,
)

In [ ]:
hits[0]

ScoredPoint(id=10, version=0, score=0.7584870735007198, payload={'title': 'الصين', 'text': 'تمثل لغة قومية هان (الصينية المنطوقة والمكتوبة) اللغة الرسمية للبلاد، وهي تستخدم في كافة أنحائها، ومن حيث تعدادها تحتل هذه اللغة المرتبة الأولى في العالم، وتسمى محليًا «زونغ ون» (中文). رغم أن اللغة الصينية تشمل أكثر من 30 ألف مقطع (أو رمز) إلا أنه وحسب إحصاء المقاطع الصينية المكتوبة في الكتب والصحف الحديثة في الوقت الحاضر، يشكل حوالي 3000 مقطع صيني 99% من نسبة المقاطع الصينية المكتوبة المتكررة. ومن بين الخمسة وخمسون أقلية قومية، فإن قوميتا هوي والمانتشو تستخدمان اللغة الهانية، بينما تستخدم كل من القوميات الثلاثة وخمسون الأخرى لغتها الخاصة، ولإحدى وعشرين أقلية قومية لغتها المكتوبة. يتم تدريس اللغات القومية في المدارس والمناطق التي تسكنها هذه الأقليات.'}, vector=None, shard_key=None, order_value=None)

In [ ]:
hits_texts = []
for rec in hits:
    hits_texts.append(rec.payload["text"])

In [ ]:
query = "من كم مقطع تتألف اللغة في الصين؟"

sentence_pairs = [[query, doc] for doc in hits_texts]

scores = reranker_model.compute_score(sentence_pairs, max_length=1024)

In [ ]:
scores

[0.8056321740150452,
 0.1250653713941574,
 0.1276526302099228,
 0.06560483574867249,
 0.04401864856481552,
 0.06853749603033066,
 0.0792103186249733,
 0.03358950838446617,
 0.035678550601005554,
 0.04336579889059067]